Written By:- Aadish Joshi  
Date:- April 24, 2019  
Topic:- Language Modeling with Ngrams  

➢ What are we trying to solve here?  
Using probabilistic models called N-grams to predict the next word from the previous n-1 words.  
Note that this problem is corpus (data set of the sentences) specific.  

➢ Grams refered to number of words taken into consideration. e.g. unigram means occurence of the single word in the corpus. Bigrams means predicting the next word based on 1 previous word. Trigrams mean predicting the next word based on 2 previous words and so on.

➢This method is count based. A simple approach uses count.  
Count all the unigrams. e.g. count of all the words in the corpus. 

➢ We’ll call a statistical model that can assess this probability a Language Model 

➢ Consider following corpus  

"The lake is very big. Its water is so transparent that you can see your face clearly. Its water is so transparent that the moon appears even bigger due to reflection"  

How to estimate the probability of the word "the" given previous words "its water is so transparent that"  

We calculate    

Count(its water is so transparent that the) = 1
and  
Count(its water is so transparent that) = 2

Hence  

P(the | its water is so transparent that ) = Count(its water is so transparent that the) / Count(its water is so transparent that)  

Unfortunately, for most sequences and for most text collections we won’t get good estimates from this method.  
➢ What we’re likely to get is 0. Or worse 0/0.  

➢ Let’s use the chain rule of probability  
➢And a particularly useful independence assumption.  

Chain rule of Probability.
P(A,B,C,D) = P(A)P(B|A)P(C|A,B)P(D|A,B,C)  

Independence Assumption.(Markov Assumption)  
That is, the probability in question is independent of its earlier history.